# Advanced Portfolio Optimization using cvxpy

## Imports

In [2]:
import cvxpy as cvx
import numpy as np

## What's our objective?
http://www.cvxpy.org/

Let's see how we can use optimization to meet a more advanced objective.  We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.


## Hints

### x vector
To create a vector of M variables $\mathbf{x} = \begin{bmatrix}
x_1 &...& x_M
\end{bmatrix}
$
we can use `cvx.Variable(m)`

### covariance matrix
If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [numpy.cov](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$), do you think this gives more priority to minimizing the difference, or minimizing the variance?

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?

### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

## Quiz

In [4]:
def optimize_portfolio(returns, index_weights, scale=.00001):
    """
    Create a function that takes the return series of a set of stocks, the index weights,
    and scaling factor. The function will minimize a combination of the portfolio variance
    and the distance of its weights from the index weights.
    The optimization will be constrained to be long only, and the weights should sum to one.

    Parameters
    ----------
    returns : numpy.ndarray
        2D array containing stock return series in each row.

    index_weights : numpy.ndarray
        1D numpy array containing weights of the index.

    scale : float
        The scaling factor applied to the distance between portfolio and index weights

    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the stocks in the optimized portfolio
    """
    # Use cvxpy to determine the weights on the assets
    # that minimizes the combination of portfolio variance and distance from index weights

    # number of stocks m is number of rows of returns, and also number of index weights
    m = len(index_weights)

    # covariance matrix of returns
    cov = np.cov(returns)

    # x variables (to be found with optimization)
    x = cvx.Variable(m)

    # portfolio variance, in quadratic form 𝜎2𝑝=𝐱𝐓𝐏𝐱
    portfolio_variance = cvx.quad_form(x,cov)

    # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = cvx.norm(x-index_weights)

    # objective function
    objective = cvx.Minimize(portfolio_variance + scale * distance_to_index)

    # constraints
    constraints = [x >= 0, sum(x) == 1]

    # use cvxpy to solve the objective
    cvx.Problem(objective, constraints).solve()

    # retrieve the weights of the optimized portfolio
    return x.value

In [5]:
"""Test with a 3 simulated stock return series"""
days_per_year = 252
years = 3
total_days = days_per_year * years

return_market = np.random.normal(loc=0.05, scale=0.3, size=days_per_year)
print(f'return_market:\n{return_market}')
return_1 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_2 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_3 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
returns = np.array([return_1, return_2, return_3])
print(f'returns is:\n{returns}')

"""simulate index weights"""
index_weights = np.array([0.9,0.15,0.05])

"""try out your optimization function"""
x_values = optimize_portfolio(returns, index_weights, scale=.00001)

print(f"The optimized weights are {x_values}, which sum to {sum(x_values):.2f}")

return_market:
[ 0.04479191  0.11652702 -0.23974862  0.07356468  0.19823349 -0.20254523
  0.51510374  0.43372708 -0.03286303 -0.60644189 -0.07904721 -0.60341853
  0.10682898  0.08780625  0.17008098  0.64784897 -0.08893558  0.41956244
  0.63654633 -0.05372626  0.48223313 -0.10061696 -0.33852636  0.3613445
  0.21472904  0.17660285  0.16589119  0.21018932  0.04244052 -0.08824958
  0.09774892  0.03646546  0.46346106  0.17378339  0.66266253  0.06629283
  0.06603085  0.31251269 -0.03384935 -0.49447409 -0.03113304  0.35257451
  0.45726227  0.28376511  0.17042101  0.58457202  0.45930145 -0.12281012
  0.26078486  0.00372    -0.07294115 -0.37421305 -0.08376597 -0.60484505
  0.38313183  0.35062344  0.29072652  0.06129167 -0.37897591 -0.34520137
  0.16503705  0.59807683  0.18251833  0.03698341  0.04110689 -0.00110662
 -0.02887413  0.01625592 -0.14350718 -0.06181422 -0.20153655  0.36591968
 -0.34420605 -0.39244814 -0.23588359  0.2428326  -0.24695245 -0.13032626
  0.29459703 -0.13982512  0.15567648 

If you're feeling stuck, you can check out the solution [here](m3l4_cvxpy_advanced_solution.ipynb)